# Introducción
## Objetivo
Este programa se tuiliza para asignar grupos a estudiantes siguiendo las prioridades:
* Todos los estudiantes (o la mayor cantidad posible), deberían ser asignados a un grupo que hayan marcado como deseable
* Los estudiantes que presentan certificado laboral/deportivo recibem prioridad al momento de la asignación grupos

## Resultado
El programa genera como archivo de salida una tabla en la que se indica el grupo asignado a cada estudiante. En una columna adicional se indica si el turno asignado se encuentra entre los elegidos por el estudiante.

## Reglas
### Sobre los grupos
* Existe un numero $n$ de grupos
* Cada grupo $n_i$ tiene un numero determinado de cupos $c_i$
* Los grupos estan agregados en $S$ *supergrupos* que pueden corresponder a turnos (Matutino, Vespertino, Nocturno), salones de clase, centros de estudio, etc. Cada grupo $n_i$ pertenece a un único supergrupo $S_j$

### Sobre los estudiantes
* Existen $m$ estudiantes. Identificados por un número de cedula o pasaporte.
* Cada estudiante debe elegir al menos 1 grupo en al menos 2 supergrupos diferentes (Todos los estudiantes presentan como mínimo 2 opciones de grupo y 2 opciones de supergrupo).
* Los estudiantes pueden presentar certificado laboral o deportivo. En este caso los estudiantes tienen prioridad en la selección de grupos

# Parametros
A continuación se detallan los parametros necesarios para ejecutar el programa. Algunos de estos pueden tener un valor por defecto.

In [60]:
# Parametros del programa
# Parametros de los grupos
archivo_grupos = 'Grupos_Disponibles.txt'
columna_supergrupos = 'Turno'
columna_min_estudiantes = 'Min Estudiantes'
columna_max_estudiantes = 'Max Estudiantes'

# Parametros de los estudiantes
archivo_estudiantes = 'Formulario_Estudiantes.txt'
columnas_supergrupos = 'Grupos mañana___Grupos tarde___Grupos noche'
columnas_certificados = 'certificado'
columnas_identificacion = 'Cedula___pasaporte'

In [2]:
# Modulos
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

In [3]:
grupos = pd.read_csv(archivo_grupos, sep='\t', index_col=0)
grupos.head()

,Turno,Min Estudiantes,Max Estudiantes
ID grupo,,,
1,Turno 1,1,35
2,Turno 1,1,35
3,Turno 1,1,35
4,Turno 1,1,35
5,Turno 1,1,35


In [69]:
estudiantes = pd.read_csv(archivo_estudiantes, sep='\t')
estudiantes.head()

,Marca temporal,Correo,Cedula,pasaporte,Nacionali,NOMBRE completo,APELLIDOS,Fecha nacimiento,depto 2023,localidad 2024,barrio,udelar previa,otros estud,Grupos mañana,Grupos tarde,Grupos noche,TRABAJA,Deporte,certificado
0,26/02/2024 12:48:29,florencialado01@gmail.com,5165495,NaN,Uruguaya,Florencia,Lado Casaglia,03/04/2002,CANELONES,Canelones La Floresta,No vivo en montevideo,NO,NO,NaN,27,NaN,No,NO,NaN
1,26/02/2024 14:04:46,florencialado01@gmail.com,5165495,NaN,Uruguaya,Florencia,Lado Casaglia,03/04/2002,CANELONES,La floresta canelones,No vivo en montevideo,SI,NO,NaN,14,NaN,No,NO,NaN
2,26/02/2024 15:59:44,diegoodera9@gmail.com,44281309,NaN,Uruguaya,Diego Maximiliano,Odera Piñeyro,14/02/1992,MONTEVIDEO,Montevideo,Guaycuru 2884(barrio Reducto),SI,NO,NaN,9,15,SI,NO,https://drive.google.com/open?id=1A4_v2vYuuRva...
3,27/02/2024 21:53:05,diegoodera9@gmail.com,44281309,NaN,Uruguayo,Diego Maximiliano,Odera Piñeyro,14/02/1992,MONTEVIDEO,Montevideo,Reducto,SI,NO,NaN,66,15,SI,NO,https://drive.google.com/open?id=1VlUhv9N7JFFB...
4,26/02/2024 15:18:35,claumansilla46197@gmail.com,46197390,NaN,Oriental,Claudia Mariana,Mansilla Goicoechea,24/08/1993,MALDONADO,Maldonado,No,SI,NO,1,9,15,SI,NO,NaN


In [5]:
# parsear argumentos
super_group = columna_supergrupos
max_stds_col = columna_max_estudiantes
min_stds_col = columna_min_estudiantes

super_group_cols = columnas_supergrupos.split('___')
cert_cols = columnas_certificados.split('___')
id_cols = columnas_identificacion.split('___')

In [5]:
def preprocess_groups(file, super_group_col, min_students_col, max_students_col):
    groups = pd.read_csv(file, sep='\t', index_col=0)
    groups['SG'] = grupos[super_group_col].replace({val:idx for idx, val in enumerate(grupos.Turno.unique())})
    
    # ensure min and max student columns have standard names
    groups['min_students'] = groups[min_students_col]
    groups['max_students'] = groups[max_students_col]
preprocess_groups(grupos, columna_supergrupos, columna_min_estudiantes, columna_max_estudiantes)
grupos

/tmp/ipykernel_5618/681537370.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  groups['SG'] = grupos[super_group_col].replace({val:idx for idx, val in enumerate(grupos.Turno.unique())})


,Turno,Min Estudiantes,Max Estudiantes,SG,min_students,max_students
ID grupo,,,,,,
1,Turno 1,1,35,0,1,35
2,Turno 1,1,35,0,1,35
3,Turno 1,1,35,0,1,35
4,Turno 1,1,35,0,1,35
5,Turno 1,1,35,0,1,35
...,...,...,...,...,...,...
54,Turno 3,1,35,2,1,35
69,Turno 3,1,35,2,1,35
70,Turno 3,1,35,2,1,35


In [21]:
# load data
def preprocess_groups(file, sg_col, min_students_col, max_students_col):
    groups = pd.read_csv(file, sep='\t', index_col=0)
    groups['SG'] = groups[sg_col].replace({val:idx for idx, val in enumerate(groups.Turno.unique())})
    
    # ensure min and max student columns have standard names
    groups['min_students'] = groups[min_students_col]
    groups['max_students'] = groups[max_students_col]
    return groups

def preprocess_students(file, id_cols):
    students = pd.read_csv(file, sep='\t')
    
    id_cols = id_cols.split('___')
    
    # identify ID col for each students    
    students['ID'] = ''
    students['Doc_type'] = ''

    for col in id_cols[::-1]:
        col_ids = students.loc[~students[col].isna(), col]
        students.loc[col_ids.index, 'ID'] = col_ids.values
        students.loc[col_ids.index, 'Doc_type'] = col
    # format ids, remove dots and dashes
    students['ID'] = students.ID.apply(lambda x : re.sub('[.]', '', re.sub('-.*', '', x))).apply(lambda x : x[:7])
    return students

# get group demand
def build_demand_table(students, groups, sg_cols):
    sg_cols = sg_cols.split('___')
    uniq_students = students.ID.unique()
    demand_tab = pd.DataFrame(False, index=uniq_students, columns=groups.index)
    sg_demand_tab = pd.DataFrame(0, index=uniq_students, columns=groups.SG.unique())
    
    # mark groups demanded by each student
    for ID, subtab in students.groupby('ID'):
        # merge all group options
        student_opts = ','.join(subtab[sg_cols].fillna('').agg(','.join, axis=1).values)
        # remove weird values, leading commas, empty spaces
        student_opts = re.sub('^,', '', re.sub(',$', '', re.sub(',,+', ',', re.sub(' ','', student_opts))))
        # generate options array
        try:
            student_opts = np.unique(student_opts.split(',')).astype(int)
            # mark student options
            demand_tab.loc[ID, student_opts] = True
        except:
            pass

    # remove weird groups (error when registering option number)
    weird_groups = demand_tab.drop(columns=groups.index)
    demand_tab = demand_tab[groups.index]

    # fill sg demand tab
    for sg, subtab in groups.groupby('SG'):
        sg_demand_tab[sg] = demand_tab[subtab.index].sum(axis=1)
    return demand_tab, sg_demand_tab, weird_groups

# build priority series
def get_certified_students(students, cert_cols):
    uniq_students = students.ID.unique()
    # establish priority students (value 0 so they appear first when sorting by priority)
    certified = pd.Series(False, index=uniq_students)
    certified[students.loc[~students[cert_cols].isna().values, 'ID'].unique()] = True
    return certified

def get_infringing_students(sg_demand, certified, min_groups, min_sg):
    infractions = (sg_demand > min_groups).sum(axis=1) < min_sg
    infractions = infractions & ~certified
    infractions = infractions.loc[infractions].index
    return infractions

def get_no_group_students(demand):
    no_group_students = (demand.sum(axis=1) == 0)
    no_group_students.loc[no_group_students].index
    return no_group_students

def build_priority(students, cert_cols, demand, sg_demand, min_groups=2, min_sg=2):
    certified = get_certified_students(students, cert_cols)
    priority = certified.apply(lambda x : 0 if x else 1)
    
    priority.loc[get_infringing_students(sg_demand, certified, min_groups, min_sg)] = 2
    priority.loc[get_no_group_students(demand)] = 3
    return priority

# assign students
def double_sorting(demand_tab, groups, priority):
    # calculate max iterations (size of largest group)
    max_iterations = groups.loc[demand_tab.columns].max_students.max()
    # prepare placement table
    placement_tab = pd.DataFrame(False, index=demand_tab.index, columns=demand_tab.columns)

    # count student options and detect certified
    student_data = demand_tab.sum(axis=1).to_frame(name='options')
    student_data['priority'] = priority
    # sort by certification status & options
    student_data.sort_values(['priority', 'options'])

    def sort_groups(dtab):
        # sort demand table by group demand, filter out depleted groups
        group_demand = dtab.sum(axis=0).sort_values()
        group_demand = group_demand[group_demand > 0]
        dtab_out = dtab[group_demand.index].copy()
        return dtab_out
    demand_tab2 = sort_groups(demand_tab)

    # iterate
    for i in range(max_iterations):
        # always start placing by the least required groups
        for grp in demand_tab2.columns:
            # check that group hasn't been depleted in the current iteration
            if demand_tab2[grp].sum() > 0:
                grp_student = demand_tab2.index[np.argmax(demand_tab2[grp])] # select the most prioritary student for the current group (certified & least options)
                # place selected student and remove them from the demand table
                placement_tab.loc[grp_student, grp] = True
                demand_tab2.drop(index=grp_student, inplace=True)
        # recalculate demand, resort groups by least demand, drop not demanded groups
        demand_tab2 = sort_groups(demand_tab2)

    # announce total unplaced students
    total_students = demand_tab.shape[0]
    placed_students = placement_tab[placement_tab.any(axis=1)].index.values
    unplaced_students = demand_tab2
    return placement_tab, unplaced_students

# accomodate unplaced students
def transfer(placement_tab, to_transfer, max_students=35):
    # locate available places for students to transfer
    avail_places = max_students - placement_tab[to_transfer.columns].sum(axis=0)
    avail_places = avail_places[avail_places > 0].sort_values(ascending=False)

    # sort students by options (ascending)
    to_transfer = to_transfer.iloc[np.argsort(to_transfer.sum(axis=1))].copy()
    # arrange transfers
    transfer_series = pd.Series(-1, index=to_transfer.index)
    for grp, places in avail_places.items():
        group_students = to_transfer[grp]
        group_students = group_students.loc[group_students].iloc[:places].index.values
        transfer_series[group_students] = grp
        to_transfer.drop(index=group_students, inplace=True)
    transfer_series = transfer_series[transfer_series > -1]
    
    # move students
    placement_tab2 = placement_tab.copy()
    placement_tab2.loc[transfer_series.index] = False
    for student, grp in transfer_series.items():
        placement_tab2.loc[student, grp] = True
    return placement_tab2

def get_moveable_students(placement_tab, demand_tab, max_students=35):
    student_counts = placement_tab.sum(axis=0)
    full_groups = student_counts[student_counts == max_students].index.values
    students_in_full = placement_tab[placement_tab[full_groups].any(axis=1)].index.values

    moveable_students = demand_tab.loc[students_in_full].drop(columns=full_groups)
    moveable_students = moveable_students.loc[moveable_students.any(axis=1), moveable_students.any(axis=0)]
    return moveable_students

def accomodate(placement_tab, unplaced_students, demand_tab, max_students=35):
    # move unplaced students into the freed places
    moveable = get_moveable_students(placement_tab, demand_tab, max_students)

    # transfer moveable students
    placement_tab2 = transfer(placement_tab, moveable, max_students)

    # place unplaced students
    placement_tab2 = transfer(placement_tab2, unplaced_students, max_students)
    unplaced2 = placement_tab2.loc[placement_tab.any(axis=1)]
    return placement_tab2, unplaced2

In [91]:
def main(students_file,
         groups_file,
         id_cols,
         sg_students,
         sg_groups,
         min_students_col,
         max_students_col,
         cert_cols,
         min_groups=2,
         min_sg=2):
    # load data
    students = preprocess_students(students_file, id_cols)
    groups = preprocess_groups(groups_file, sg_groups, min_students_col, max_students_col)

    # build demand
    demand, sg_demand, weird_demand = build_demand_table(students, groups, sg_students)

    # build priority
    priority = build_priority(students, cert_cols, demand, sg_demand, min_groups, min_sg)

    # distribute
    placement, unplaced = double_sorting(demand, groups, priority)

    # acomodate
    placement2, unplaced2 = accomodate(placement, unplaced, demand, max_students=groups.max_students.max())
    return placement2, unplaced2
    

In [10]:
# load data
students_file = 'Formulario_Estudiantes.txt'
id_cols = 'Cedula___pasaporte'
students = preprocess_students(students_file, id_cols)
students.head()

,Marca temporal,Correo,Cedula,pasaporte,Nacionali,NOMBRE completo,APELLIDOS,Fecha nacimiento,depto 2023,localidad 2024,...,udelar previa,otros estud,Grupos mañana,Grupos tarde,Grupos noche,TRABAJA,Deporte,certificado,ID,Doc_type
0,26/02/2024 12:48:29,florencialado01@gmail.com,5165495,NaN,Uruguaya,Florencia,Lado Casaglia,03/04/2002,CANELONES,Canelones La Floresta,...,NO,NO,NaN,27,NaN,No,NO,NaN,5165495,Cedula
1,26/02/2024 14:04:46,florencialado01@gmail.com,5165495,NaN,Uruguaya,Florencia,Lado Casaglia,03/04/2002,CANELONES,La floresta canelones,...,SI,NO,NaN,14,NaN,No,NO,NaN,5165495,Cedula
2,26/02/2024 15:59:44,diegoodera9@gmail.com,44281309,NaN,Uruguaya,Diego Maximiliano,Odera Piñeyro,14/02/1992,MONTEVIDEO,Montevideo,...,SI,NO,NaN,9,15,SI,NO,https://drive.google.com/open?id=1A4_v2vYuuRva...,4428130,Cedula
3,27/02/2024 21:53:05,diegoodera9@gmail.com,44281309,NaN,Uruguayo,Diego Maximiliano,Odera Piñeyro,14/02/1992,MONTEVIDEO,Montevideo,...,SI,NO,NaN,66,15,SI,NO,https://drive.google.com/open?id=1VlUhv9N7JFFB...,4428130,Cedula
4,26/02/2024 15:18:35,claumansilla46197@gmail.com,46197390,NaN,Oriental,Claudia Mariana,Mansilla Goicoechea,24/08/1993,MALDONADO,Maldonado,...,SI,NO,1,9,15,SI,NO,NaN,4619739,Cedula


In [14]:
groups_file = 'Grupos_Disponibles.txt'
sg_groups = 'Turno'
min_students_col = 'Min Estudiantes'
max_students_col = 'Max Estudiantes'
groups = preprocess_groups(groups_file, sg_groups, min_students_col, max_students_col)
groups.head()

/tmp/ipykernel_5580/2543390740.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  groups['SG'] = groups[sg_col].replace({val:idx for idx, val in enumerate(groups.Turno.unique())})


,Turno,Min Estudiantes,Max Estudiantes,SG,min_students,max_students
ID grupo,,,,,,
1,Turno 1,1,35,0,1,35
2,Turno 1,1,35,0,1,35
3,Turno 1,1,35,0,1,35
4,Turno 1,1,35,0,1,35
5,Turno 1,1,35,0,1,35


In [17]:
# build demand
sg_students = 'Grupos mañana___Grupos tarde___Grupos noche'
demand, sg_demand, weird_demand = build_demand_table(students, groups, sg_students)
display(demand.head())
display(sg_demand.head())
display(weird_demand.head())

ID grupo,1,2,3,4,5,6,7,8,19,20,...,35,36,51,52,53,54,69,70,71,72
5165495,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4428130,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4619739,True,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,True,False,False
4626722,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4852745,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,False,False,True,False,True


,0,1,2
5165495,0,2,0
4428130,0,2,1
4619739,3,3,3
4626722,0,1,1
4852745,0,0,6


ID grupo,2007,2003,2006,2008
5165495,NaN,NaN,NaN,NaN
4428130,NaN,NaN,NaN,NaN
4619739,NaN,NaN,NaN,NaN
4626722,NaN,NaN,NaN,NaN
4852745,NaN,NaN,NaN,NaN


In [18]:
# build priority
cert_cols = 'certificado'
priority = build_priority(students, cert_cols, demand, sg_demand, min_groups=2, min_sg=2)
priority.head()

5165495    2
4428130    0
4619739    1
4626722    2
4852745    0
dtype: int64

In [19]:
# distribute
placement, unplaced = double_sorting(demand, groups, priority)
display(placement.head())
display(unplaced.head())

ID grupo,1,2,3,4,5,6,7,8,19,20,...,35,36,51,52,53,54,69,70,71,72
5165495,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4428130,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4619739,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
4626722,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4852745,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


ID grupo,9,1,14,24,13,38
5501866,False,False,False,False,False,False
5582817,False,False,False,False,False,True
5583272,False,False,False,False,False,False
5584101,False,False,False,True,False,True
5590518,False,False,False,False,True,True


In [22]:
# acomodate
placement2, unplaced2 = accomodate(placement, unplaced, demand, max_students=groups.max_students.max())
display(placement2.head())
display(unplaced2.head())

ID grupo,1,2,3,4,5,6,7,8,19,20,...,35,36,51,52,53,54,69,70,71,72
5165495,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4428130,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4619739,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
4626722,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4852745,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


ID grupo,1,2,3,4,5,6,7,8,19,20,...,35,36,51,52,53,54,69,70,71,72
5165495,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4428130,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4619739,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
4626722,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4852745,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [33]:
def summarize_placement(placement_tab):
    students_per_group = placement_tab.sum(axis=0).sort_values().to_frame().T
    unplaced_students = placement_tab.loc[~placement_tab.any(axis=1)].index.to_numpy()
    return students_per_group, unplaced_students

spg, unp = summarize_placement(placement)
spg2, unp2 = summarize_placement(placement2)
display(spg)
display(unp)
display(spg2)
display(unp2)

ID grupo,70,69,53,71,54,52,72,35,34,33,...,10,8,6,2,38,9,13,14,24,1
0,18,18,18,18,19,19,20,21,21,21,...,26,26,26,26,35,35,35,35,35,35


array(['5501866', '5582817', '5583272', '5584101', '5590518', '5590699',
       '5591698', '5594324', '5596694', '5598626', '5599080', '5599364',
       '5599708', '5600090', '5600091', '5600207', '5602140', '5602780',
       '5603216', '5603447', '5604121', '5605094', '5606803', '5607923',
       '5608825', '5609277', '5610317', '5617328', '5618950', '5619638',
       '5620185', '5621049', '5624216', '5624771', '5625240', '5625990',
       '5626727', '5627379', '5629030', '5643575', '5645730', '5646480',
       '5648289', '5653261', '5656047', '5656576', '5657282', '5658400',
       '5660843', '5664211', '5668482', '5671630', '5684382', '5688984',
       '5702135', '5710186', '5710560', '5729855', '5736058', '5744633',
       '5772398', '5938029', '6020041', '6367785', '6372219', '6391570',
       '6440043', '6464877', '6542873', '6595648', '6623958', '6644785',
       '5601410'], dtype=object)

ID grupo,70,69,53,71,54,52,72,35,34,33,...,10,8,36,2,9,13,24,38,14,1
0,18,18,18,18,19,19,20,21,21,21,...,27,27,27,31,35,35,35,35,35,35


array(['5501866', '5583272', '5584101', '5590699', '5594324', '5596694',
       '5599080', '5599364', '5599708', '5600090', '5600091', '5600207',
       '5602140', '5602780', '5603216', '5603447', '5604121', '5605094',
       '5606803', '5607923', '5608825', '5609277', '5610317', '5617328',
       '5618950', '5619638', '5620185', '5621049', '5624216', '5624771',
       '5625240', '5625990', '5626727', '5627379', '5653261', '5656047',
       '5656576', '5657282', '5668482', '5671630', '5684382', '5688984',
       '5710186', '5710560', '5729855', '5736058', '5744633', '5772398',
       '5938029', '6020041', '6367785', '6372219', '6391570', '6440043',
       '6464877', '6595648', '6623958', '5601410'], dtype=object)